# 0.0  Arithmatic encoding:: Introduction

<img src = "https://www.researchgate.net/profile/Mahnaz-Sinaie/publication/47696414/figure/fig2/AS:667822808121349@1536232773304/An-example-of-arithmetic-coding-the-source-symbols-are-a-b-c-with-pa-0011-pb.png" width="400px" align="right">

Arithmatic encoding is used mainly for encoding stream of bits, without the need to encode a complete bunch of of all combinations of N bits. It depends on the idea that stream of bits that has low probability of occurance, will hav greater number of bits, and high frequency stream will small number of bits

# Visualizer

for visualising the list probabilites, **please ignore it**; not relavant to the arithmatic encding

In [ ]:
import copy

In [ ]:
def visulize_probs(probs: list) -> None:
  print("*"*20)
  probs_new = copy.deepcopy(probs)[::-1]
  max_length = 10
  for i,p in enumerate(probs_new) :
    print("*{:.4f}".format(p))
    if i+1 == len(probs_new):
      break
    print("|\n"*(max(1, int(max_length*((probs_new[i]-probs_new[i+1])/(probs_new[0]-probs_new[-1]))))))
  print("*"*20)

# 1.0 Encoder
To encode:
1. Get Lower(L) and Higher(H) probability bounds of the stream
2. split probabilities (add more bits) untill:

L<= m1

H> m2
 where 

 
$
 *-----------------*m2 \\
 |\hspace{9cm}      |\\
 *-----------------*m1\\
 |\hspace{9cm}      |\\
 *------------------* \\
 $

 First step in Arthmatic encoding is to split the whole range between 0, and 1 according to probabiliy of each symbol for example, if we have list of symbols of [['u', 'o', 'i', 'e', 'a'] of probabilities [0.07, 0.3, 0.09, 0.42, 0.12]

*1

|

|

*0.88

|

|
*0.46

|

|

*0.37

 |

 |

*0.07

 |

 *0

In [ ]:
def accumulate_lst(symbols_probs: list, start_prob: float = 0, end_prob: float = 1, verbose = True)-> list:
    """
    Accumulate the probabilites based on given on porbabilites of the symbols
    Arguments:
      symbols_probs(list): probability of each symbol
      start_prob(float): bottom probabilty (intiallty is set to 0)
      end_prob(float): upper probabilty (intiallty is set to 1)
    """
    acc_probs = []
    new_point = start_prob
    acc_probs.append(start_prob) # adding bootom probability at the end of the acc_list(at first it should be zero)
    for i in range(len(symbols_probs)): # accumulate probs
      new_range = symbols_probs[i]*(end_prob-start_prob) # get the range of of symbol from given probability
      new_point = new_point + new_range
      acc_probs.append(new_point) ## append it to list
    if verbose:
      visulize_probs(acc_probs)
    return acc_probs
_ = accumulate_lst([0.07, 0.3, 0.09, 0.42, 0.12])

********************
*1.0000
|

*0.8800
|
|
|
|

*0.4600
|

*0.3700
|
|
|

*0.0700
|

*0.0000
********************


For a given stream of data like "iou", we must calculate range of probabilities it has to be in.

for that to habben, for example inital distribution is as follows:
*1

|a

*0.88

|e

*0.46

|i

*0.37

 |o


*0.07

 |u

 *0

 and first letter is "i"
*0.4600

|

*0.4492

|

*0.4114

|

*0.4033

|

*0.3763

|

*0.3700

and so on

In [ ]:
def get_bounds(word: str, symbols: list, probs: list, start_prob: float = 0, end_prob: float = 1, verbose = True):
  if word == "":  
    return start_prob, end_prob
  first_char = word[0]
  index_char = symbols.index(first_char)
  acc_probs = accumulate_lst(probs, start_prob, end_prob, verbose = verbose  )
  return get_bounds(word = word[1:], symbols = symbols, probs = probs, start_prob = acc_probs[index_char], end_prob = acc_probs[index_char+1], verbose = verbose)
get_bounds("iou", ['u', 'o', 'i', 'e', 'a'], [0.07, 0.3, 0.09, 0.42, 0.12] )

********************
*1.0000
|

*0.8800
|
|
|
|

*0.4600
|

*0.3700
|
|
|

*0.0700
|

*0.0000
********************
********************
*0.4600
|

*0.4492
|
|
|
|

*0.4114
|

*0.4033
|
|

*0.3763
|

*0.3700
********************
********************
*0.4033
|

*0.4001
|
|
|
|

*0.3887
|

*0.3863
|
|
|

*0.3782
|

*0.3763
********************


(0.37629999999999997, 0.37818999999999997)

### Arithmatic encoding

first for stream of data to fall in range these conditions must be satisfied

L<= m1

H> m2
 where 

 
$
 *-----------------*m2 \\
 |\hspace{9cm}      |\\
 *-----------------*m1\\
 |\hspace{9cm}      |\\
 *------------------* \\
 $

In [ ]:
def check(L,H, m1, m2):
    return L <= m1 and H > m2

For arithmatic encoding, I used recursion in the implmentation

for example, stream is iou, and symbol probability is 0.37, and 0.42.
1- first i split prbability between 0, 1,

2- symbol prob is in the lower bound

3- expand lower band by adding zeroa and i


For the

In [ ]:
L, H = get_bounds("iou", ['u', 'o', 'i', 'e', 'a'], [0.07, 0.3, 0.09, 0.42, 0.12],verbose = True )

********************
*1.0000
|

*0.8800
|
|
|
|

*0.4600
|

*0.3700
|
|
|

*0.0700
|

*0.0000
********************
********************
*0.4600
|

*0.4492
|
|
|
|

*0.4114
|

*0.4033
|
|

*0.3763
|

*0.3700
********************
********************
*0.4033
|

*0.4001
|
|
|
|

*0.3887
|

*0.3863
|
|
|

*0.3782
|

*0.3763
********************


In [ ]:
def encode_arithmatic(L,H, bottom = 0 , upper = 1, code_down = "0", code_up = "1", verbose = True):
    """
    L: Lower bound of stream
    H: Higher bound of stream
    bottom, upper, code_down, code_up: are used by recuresion, please don't use them
    """
    middle = (bottom+upper)/2 # middle probabilty
    if verbose: print("**************L= {:.4f},H= {:.4f} \n|-------------|*{:.4f}\n| {} |\n|-------------|*{}\n| {} |\n|-------------|*{:.4f}\n**************".format(L,H,upper,code_up, middle,code_down,bottom))
    if check(L, H, bottom, middle): # does stream reside in bottom band?
      print("It's in the bottom at m1={}, and m2= {}, where the code is".format(bottom,middle, code_down))
      return code_down
    if check(L, H, middle, upper): # does stream reside in upper band?
      print("It's in the upper at m1={:.4f}, and m2= {:.4f}, where the code is {} ".format(middle,upper, code_down))
      return code_up
    if L >= middle: # if probabilty resided in bottom band, expand bottom, and let code down = code down + 0, same for up and append 1. and modify probability
      print("Expand Bottom\n")
      return encode_arithmatic(L, H, bottom = middle, upper = upper, code_down = code_up + "0", code_up = code_up + "1")
    else:
      print("Expand Upper\n")
      return encode_arithmatic(L, H, bottom = bottom, upper = middle, code_down = code_down + "0", code_up = code_down + "1")




_ = encode_arithmatic(L, H )

**************L= 0.3763,H= 0.3782 
|-------------|*1.0000
| 1 |
|-------------|*0.5
| 0 |
|-------------|*0.0000
**************
Expand Upper

**************L= 0.3763,H= 0.3782 
|-------------|*0.5000
| 01 |
|-------------|*0.25
| 00 |
|-------------|*0.0000
**************
Expand Bottom

**************L= 0.3763,H= 0.3782 
|-------------|*0.5000
| 011 |
|-------------|*0.375
| 010 |
|-------------|*0.2500
**************
Expand Bottom

**************L= 0.3763,H= 0.3782 
|-------------|*0.5000
| 0111 |
|-------------|*0.4375
| 0110 |
|-------------|*0.3750
**************
Expand Upper

**************L= 0.3763,H= 0.3782 
|-------------|*0.4375
| 01101 |
|-------------|*0.40625
| 01100 |
|-------------|*0.3750
**************
Expand Upper

**************L= 0.3763,H= 0.3782 
|-------------|*0.4062
| 011001 |
|-------------|*0.390625
| 011000 |
|-------------|*0.3750
**************
Expand Upper

**************L= 0.3763,H= 0.3782 
|-------------|*0.3906
| 0110001 |
|-------------|*0.3828125
| 011

# Decoder

you must specify number of symboles

In [ ]:
def decode_arithmatic(code, symbols: list, probs: list,max_length, start_prob = 0 , end_prob = 1, output_char = "", ):
  """
    code: code to decode
    symbols: symbols that you have a probabiliy for
    probs: probabiliy of each symbol
    max: length: must be speicifed for decoding
    start_prob, end_prob, output_char: used by recursion please don't use them

  """
  if len(output_char) == max_length: # You must specify number of symbols
    return output_char
  prob = sum([2**(-(i+1)) for i in range(len(code)) if code[i] == "1"]) # converty binary code into fraction as probability
  acc_probs = accumulate_lst(probs, start_prob, end_prob  ) # accumulate the probabilty for finding the range
  index = 0
  while True: # finding the range of probabilty for the symbol
    if prob >= acc_probs[index] and prob < acc_probs[index+1]:
      break
    index += 1
  output_char  =  output_char + symbols[index]
  return decode_arithmatic(code, symbols = symbols, probs = probs,max_length = max_length, start_prob = acc_probs[index] , end_prob = acc_probs[index+1], output_char = output_char) # find next char
  # print(prob, index, acc_probs)
    
    

decode_arithmatic("01100000011",['u', 'o', 'i', 'e', 'a'], [0.07, 0.3, 0.09, 0.42, 0.12],max_length =3)

********************
*1.0000
|

*0.8800
|
|
|
|

*0.4600
|

*0.3700
|
|
|

*0.0700
|

*0.0000
********************
********************
*0.4600
|

*0.4492
|
|
|
|

*0.4114
|

*0.4033
|
|

*0.3763
|

*0.3700
********************
********************
*0.4033
|

*0.4001
|
|
|
|

*0.3887
|

*0.3863
|
|
|

*0.3782
|

*0.3763
********************


'iou'

In [ ]:
_

'01100000011'